⚠️ Need to have .env file in the project root with OPENAI_API_KEY={your_openai_key}

In [24]:
!pip install langchain openai python-dotenv

  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)


In [1]:
import json

with open('data/tags.json', 'r') as f:
    file_tags = json.load(f)
file_tags

{'github-gradle-gcp.yaml': ['gradle', 'gcp', 'github'],
 'github-gradle-openshift.yaml': ['gradle', 'openshift', 'github'],
 'github-maven-gcp.yaml': ['maven', 'gcp', 'github'],
 'github-maven-openshift.yaml': ['maven', 'openshift', 'github'],
 'github-npm-gcp.yaml': ['npm', 'gcp', 'github'],
 'github-npm-openshift.yaml': ['npm', 'openshift', 'github'],
 'jenkins-gradle-gcp.txt': ['gradle', 'gcp', 'jenkins'],
 'jenkins-gradle-openshift.txt': ['gradle', 'openshift', 'jenkins'],
 'jenkins-maven-gcp.txt': ['maven', 'gcp', 'jenkins'],
 'jenkins-maven-openshift.txt': ['maven', 'openshift', 'jenkins'],
 'jenkins-npm-gcp.txt': ['npm', 'gcp', 'jenkins'],
 'jenkins-npm-openshift.txt': ['npm', 'openshift', 'jenkins']}

In [2]:
def search_files_by_tags(tags):
    return [file_path for file_path, ftags in file_tags.items() if all(tag in ftags for tag in tags)]

search_files_by_tags(['maven', 'openshift'])

['github-maven-openshift.yaml', 'jenkins-maven-openshift.txt']

In [11]:
def get_file_contents_by_tags(tags):
    matching_files = search_files_by_tags(tags)
    if len(matching_files) == 1:
        with open('data/' + matching_files[0], 'r') as f:
            return f.read()
    else:
        return "Error: The specified tags match more than one file."

tags = ['npm', 'gcp', 'jenkins']
example = get_file_contents_by_tags(tags)
example

"pipeline {\n    agent any\n\n    environment {\n        GCP_PROJECT = 'my-gcp-project' // Replace with your GCP project name\n        GCP_SERVICE_ACCOUNT_KEY = credentials('gcp-service-account-key') // Replace with your Jenkins credentials ID for GCP service account key\n        GCP_APP_NAME = 'my-npm-app' // Replace with your GCP App Engine application name\n    }\n\n    stages {\n        stage('Checkout') {\n            steps {\n                checkout scm\n            }\n        }\n\n        stage('Build') {\n            steps {\n                sh 'npm install'\n                sh 'npm run build'\n            }\n        }\n\n        stage('Test') {\n            steps {\n                sh 'npm test'\n            }\n        }\n\n        stage('Deploy to GCP') {\n            steps {\n                withCredentials([file(credentialsId: 'gcp-service-account-key', variable: 'GCP_KEY_FILE')]) {\n                    sh 'gcloud auth activate-service-account --key-file=$GCP_KEY_FILE'\n  

In [46]:
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

chat = ChatOpenAI(temperature = 0, openai_api_key=openai_api_key)
system_message_prompt = SystemMessagePromptTemplate.from_template_file('data/system_prompt.txt', [])
human_message_prompt = HumanMessagePromptTemplate.from_template_file('data/human_prompt.txt', input_variables = ['example']).format_messages(example = example)

chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt,
     MessagesPlaceholder(variable_name='history')] +
     human_message_prompt +
     [HumanMessagePromptTemplate.from_template("{input}")]
)

memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=chat_prompt, llm=chat)

response = conversation.predict(input = '')

print(response)

pipeline {
    agent any

    environment {
        GCP_PROJECT = 'my-gcp-project' // Replace with your GCP project name
        GCP_SERVICE_ACCOUNT_KEY = credentials('gcp-service-account-key') // Replace with your Jenkins credentials ID for GCP service account key
        GCP_APP_NAME = 'my-npm-app' // Replace with your GCP App Engine application name
    }

    stages {
        stage('Checkout') {
            steps {
                checkout scm
            }
        }

        stage('Build') {
            steps {
                sh 'npm install'
                sh 'npm run build'
            }
        }

        stage('Test') {
            steps {
                sh 'npm test'
            }
        }

        stage('Deploy to GCP') {
            steps {
                withCredentials([file(credentialsId: 'gcp-service-account-key', variable: 'GCP_KEY_FILE')]) {
                    sh 'gcloud auth activate-service-account --key-file=$GCP_KEY_FILE'
                    sh 'gcloud confi

In [47]:
print(conversation.run("""1. test-project
2. gcp-key
3. my-app"""))

pipeline {
    agent any

    environment {
        GCP_PROJECT = 'test-project'
        GCP_SERVICE_ACCOUNT_KEY = credentials('gcp-key')
        GCP_APP_NAME = 'my-app'
    }

    stages {
        stage('Checkout') {
            steps {
                checkout scm
            }
        }

        stage('Build') {
            steps {
                sh 'npm install'
                sh 'npm run build'
            }
        }

        stage('Test') {
            steps {
                sh 'npm test'
            }
        }

        stage('Deploy to GCP') {
            steps {
                withCredentials([file(credentialsId: 'gcp-key', variable: 'GCP_KEY_FILE')]) {
                    sh 'gcloud auth activate-service-account --key-file=$GCP_KEY_FILE'
                    sh 'gcloud config set project $GCP_PROJECT'
                    sh 'gcloud app deploy --version=$BUILD_NUMBER --quiet'
                }
            }
        }
    }
}

###### Questions
1. Is there any specific branc

In [48]:
print(conversation.run("""1. deployment-branch
2. SonarQube
3. Eslint"""))

pipeline {
    agent any

    environment {
        GCP_PROJECT = 'test-project' // Replace with your GCP project name
        GCP_SERVICE_ACCOUNT_KEY = credentials('gcp-key') // Replace with your Jenkins credentials ID for GCP service account key
        GCP_APP_NAME = 'my-app' // Replace with your GCP App Engine application name
    }

    stages {
        stage('Checkout') {
            steps {
                checkout scm
            }
        }

        stage('Build') {
            steps {
                sh 'npm install'
                sh 'npm run build'
            }
        }

        stage('Test') {
            steps {
                sh 'npm test'
            }
        }

        stage('Static Code Analysis') {
            steps {
                sh 'npm run eslint' // Replace with your ESLint command
            }
        }

        stage('Code Quality') {
            steps {
                withSonarQubeEnv('SonarQube') {
                    sh 'npm run sonar-scanner' // R